In [148]:
#Библиотеки
import pandas as pd
import numpy as np
import warnings
import json
import os
warnings.filterwarnings('ignore')

#Функция изъятия первых n слов
def cut_words(df, col, n):
    new = df[col].str.split(' ')
    new = new.str[0:n]
    new = new.apply(lambda x: ' '.join(x))
    return new

#Открытие json
with open('./data.json') as fp:
    data = json.load(fp)

#1-ый уровень
fields = ['TITLE', 'ITEMS']
table = pd.json_normalize(data,max_level=1)[fields]
table = table.explode('ITEMS', ignore_index=True)
table = pd.concat([table, table['ITEMS'].apply(pd.Series)], axis=1)
fields = ['TITLE', 'title', 'ITEMS', 'STATUS_RESULT']
table = table[fields]
col_replace = {'TITLE': 'project', 'title': 'stage',
               'ITEMS': 'items', 'STATUS_RESULT': 'status_stage'}
table.rename(columns = col_replace, inplace = True)
table = table.iloc[:, np.r_[0,2,4,5]]
table = table.explode('items', ignore_index=True)
table = pd.concat([table, table['items'].apply(pd.Series)], axis=1)

#2-ой уровень
fields = ['project', 'stage', 'status_stage', 'title',
          'RESPONSOBLE_FIO', 'DURATION', 'BEGIN_DATE_FORMAT',
          'CLOSE_DATE_FORMAT', 'STATUS_RESULT', 'ITEMS', 'TASKS']
table = table[fields]
col_replace = {'title':'function', 'RESPONSOBLE_FIO': 'role_function',
               'DURATION': 'duration_function', 'BEGIN_DATE_FORMAT': 'start_function',
               'CLOSE_DATE_FORMAT': 'finish_function', 'STATUS_RESULT': 'status_function',
               'ITEMS': 'items', 'TASKS': 'tasks'}
table.rename(columns = col_replace, inplace = True)

#3-й уровень
fields = ['project', 'stage', 'function', 'status_stage', 'role_function',
         'duration_function', 'start_function', 'finish_function', 'tasks']
table = table[fields]
table = table.explode('tasks', ignore_index=True)
table = pd.concat([table, table['tasks'].apply(pd.Series)], axis=1)

#Удаление лишних столбцов
fields.append('TITLE')
table = table[fields]
col_replace = {'TITLE': 'task',
               'status_stage': 'status'}
table.rename(columns = col_replace, inplace = True)
table = table.drop('tasks', axis = 1)

#Создание структуры таблицы
table['decomposition_1'] = np.where(table['stage']==' ',
                                    'Задачи договора',
                                    'Этапы договора')
table['document'] = np.where(cut_words(table, 'function', 1).str[:-1].str.isnumeric()==True,
                             table['function'],
                             ' ')
table['function'] = np.where(table['document'] != ' ',
                             ' ',
                             table['function'])
table['decomposition_2'] = np.where(table['function']==' ',
                                    np.where(table['document']!=' ',
                                             'Документооборот',
                                             table['task']),
                                    'Функции')
table['task'] = np.where(table['task'] == table['decomposition_2'],
                         ' ',
                         table['task'])
table['decomposition_3'] = np.where(table['decomposition_2'] == 'Функции',
                                    table['function'],
                                    np.where(table['decomposition_2'] == 'Документооборот',
                                             table['document'],
                                             ' '))
table['stage'] = np.where(table['decomposition_1']=='Задачи договора',
                          table['decomposition_2'],
                          table['stage'])
table['decomposition_2'] = np.where(table['decomposition_2']==table['stage'],
                                    ' ',
                                    table['decomposition_2'])

#Добавление иерархии
col_order = ['project', 'decomposition_1', 'stage',
             'decomposition_2', 'decomposition_3', 'task',
             'role_function', 'duration_function', 'start_function',
             'finish_function']
table = table[col_order]
for i in range(5):
    level = list(set(map(tuple,table.iloc[:,:(1+i)].values.tolist())))
    length = table.shape[1] - len(level[0])
    for j in level:
        row = list(j)
        table = table.append(pd.Series(row + [' ']*length,
                             index=list(table.columns)),
                             ignore_index=True)

#Составление сводной таблицы
pivot = table.pivot_table(index = col_order[:6],
                          values = col_order[6:],
                          aggfunc ='sum')
pivot = pivot[~pivot.index.duplicated(keep='first')]

#Сброс индекса
pivot.to_excel('./data.xlsx')
pivot = pd.read_excel('./data.xlsx').replace(np.nan, ' ')
os.remove('./data.xlsx')

#html-конвертация


,project,decomposition_1,stage,decomposition_2,decomposition_3,task,duration_function,finish_function,role_function,start_function
0,УВВ-08-0403-22 от 28.02.22,,,,,,,,,
1,,Этапы договора,,,,,,,,
2,,,01 Развитие сервисов Портала для потребителей ...,,,,,,,
3,,,,Документооборот,,,,,,
4,,,,,01. Отчетные документы (Описание технических р...,,,,,
...,...,...,...,...,...,...,...,...,...,...
354,,,,,4.1.2.3 МЛК. Требования к защите от ошибочных ...,,,,,
355,,,,,4.1.2.4 Требования к режимам работы,,,,,
356,,,,,4.1.8.1 Общие требования к информационной безо...,,,,,
357,,,,,4.1.8.2 МЛК. Частное техническое задание на по...,,,,,
